# LangChain Expression Language (LCEL)

A Linguagem de Expressão LangChain, ou LCEL, é uma forma declarativa de **compor cadeias de maneira fácil**. A LCEL foi projetada desde o primeiro dia para suportar a colocação de protótipos em produção, sem a necessidade de alterações no código, desde a cadeia mais simples “prompt + LLM” até as cadeias mais complexas (já vimos pessoas executando com sucesso cadeias LCEL com centenas de etapas em produção). Para destacar alguns dos motivos pelos quais você pode querer usar a LCEL:
- **Suporte a streaming de primeira classe**: menor tempo possível para saída do primeiro token produzidio;
- **Suporte assíncrono**: Qualquer cadeia construída com a LCEL pode ser chamada tanto com a API síncrona;
- **Execução paralela otimizada**: Sempre que suas cadeias LCEL tiverem etapas que podem ser executadas em paralelo, automaticamente é feito isso;
- **Retentativas e fallbacks**: É maneira de tornar suas cadeias mais confiáveis em grande escala, na qual ações alternativas podem ser tomadas no caso de um erro em uma cadeia
- **Acessar resultados intermediários**: auxiliando na depuração de uma cadeia;

## Um exemplo simples de LCEL

O exemplo mais simples que podemos mostrar seria na construção de *prompt + model*

In [1]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model= 'gpt-4o-mini')

In [3]:
from langchain_core.prompts import ChatPromptTemplate as cpt

prompt = cpt.from_template('Crie uma frase sobre o seguinte: {assunto}')

In [4]:
chain = prompt | model

In [8]:
resposta = chain.invoke({'assunto': 'Um casal de namorados Thomas e Laura'})
resposta.content

'Thomas e Laura caminhavam de mãos dadas pelo parque, vivendo momentos de pura alegria e complicidade sob o brilho do entardecer.'

### Adicinando mais elementos a chain

In [10]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [11]:
chain.invoke({'assunto': 'gatinhos'})

'Gatinhos são pequenas bolinhas de pelo que enchem nossos dias de alegria e ternura com suas travessuras.'

### ATENÇÃO! A ordem importa

In [ ]:
chain = model | prompt | output_parser

chain.invoke({'assunto': 'gatinhos'})

ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

### Como eu sei a ordem?

Para atingir o mesmo objetivo sem a criação da chain, os passos que eu deveria seguir seriam:
- Formatar o prompt template
- Enviar o prompt formatado para o modelo
- Fazer o parseamento do saída do modelo
Essa mesma ordem deve ser seguida para não termos erros

In [15]:
input = {'assunto': 'gatinhos'}
prompt_formatado = prompt.invoke(input)
prompt_formatado

ChatPromptValue(messages=[HumanMessage(content='Crie uma frase sobre o seguinte: gatinhos', additional_kwargs={}, response_metadata={})])

In [16]:
resposta = model.invoke(prompt_formatado)
resposta

AIMessage(content='Gatinhos são pequenos feixes de alegria que trazem diversão e ternura para nossos dias.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 18, 'total_tokens': 38, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run-09a94d41-232d-4bbb-ae90-8ba1b424eda1-0', usage_metadata={'input_tokens': 18, 'output_tokens': 20, 'total_tokens': 38, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [17]:
output_parser.invoke(resposta)

'Gatinhos são pequenos feixes de alegria que trazem diversão e ternura para nossos dias.'

### Para não errar a ordem

É importante entendermos que cada componente recebe um tipo de entrada e gera um tipo de saída, e estes tipos precisam casar:

| Component       | Tipo de Entrada                                    | Tipo de Saída        |
|-----------------|-----------------------------------------------------|----------------------|
| Prompt          | Dicionário                                         | PromptValue      |
| ChatModel       | String única, lista de mensagens de chat ou PromptValue | Mensagem de Chat     |
| LLM             | String única, lista de mensagens de chat ou PromptValue | String               |
| OutputParser    | A saída de um LLM ou ChatModel                     | Depende do parser    |
| Retriever       | String única                                       | Lista de Documentos  |
| Tool            | String única ou dicionário, dependendo da ferramenta| Depende da ferramenta|

Podemos verificar isso pelos output e input schemas:

In [24]:
print(prompt.input_schema.model_json_schema())

{'properties': {'assunto': {'title': 'Assunto', 'type': 'string'}}, 'required': ['assunto'], 'title': 'PromptInput', 'type': 'object'}


In [26]:
print(prompt.output_schema.model_json_schema())

{'$defs': {'AIMessage': {'additionalProperties': True, 'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.', 'properties': {'content': {'anyOf': [{'type': 'string'}, {'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}, 'type': 'array'}], 'title': 'Content'}, 'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'}, 'response_metadata': {'title': 'Response Metadata', 'type': 'object'}, 'type': {'const': 'ai', 'default': 'ai', 'title': 'Type', 'type': 'string'}, 'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Name'}, 'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Id'}, 'example': {'default': False, 'title': 'Example', 'type': 'boole

> Atenção!
O método schema_json será removido nas próximas versões do LangChain! É recomendado a utilização do método model_json_schema ao utilizar versões mais recents, da seguinte forma:

### Como eu teria feito com chains clássicas?

In [27]:
from langchain.chains.llm import LLMChain
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate as cpt

In [28]:
chain = LLMChain(
    llm=model,
    prompt=prompt,
    output_parser=output_parser
)
chain.invoke({'assunto': 'gatinhos'})

C:\Users\maris\AppData\Local\Temp\ipykernel_16608\1936627178.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


{'assunto': 'gatinhos',
 'text': 'Gatinhos são pequenas bolas de pelo que trazem alegria e amor aos corações de quem os acolhe.'}